In [1]:
######################################################
#                RUN THIS CODE BLOCK                 #
#     THIS CODE BLOCK DOES NOT NEED TO BE EDITED     #
#  DO NOT CHANGE UNLESS YOU KNOW WHAT YOU'RE DOING!! #
######################################################s
# Use this code block and the following to make sure you're outside the notebooks folder.
# This will avoid any import errors.
# You'll want the directory to be changed to the same directory containing the
# makefile, the README.md file, and the lineage folder.
# If this is the first time you're running this notebook or if you've recently restarted the
# kernel of this notebook, then %pwd should return that you're in the notebooks folder.
%pwd
######################################################
#                RUN THIS CODE BLOCK                 #
#     THIS CODE BLOCK DOES NOT NEED TO BE EDITED     #
#  DO NOT CHANGE UNLESS YOU KNOW WHAT YOU'RE DOING!! #
######################################################

'/home/staylor/lineage-growth/notebooks'

In [2]:
######################################################
#                RUN THIS CODE BLOCK                 #
#     THIS CODE BLOCK DOES NOT NEED TO BE EDITED     #
#  DO NOT CHANGE UNLESS YOU KNOW WHAT YOU'RE DOING!! #
######################################################
# If the above isn't saying that you're in the notebooks folder, 
# then there's no need to worry. The following code will automatically
# identify where you've installed the lineage-growth repository and the 
# lineage repository. It will then automatically move you to outside the 
# notebooks folder to ensure that the rest of the code in the notebook 
# works.
tHMM_loc = !find ~ -name "tHMM.py" -printf '%h' | sort -u 
lineage_growth_repo = f"{tHMM_loc[0]}/..";
%pushd $lineage_growth_repo
# Note that pushd keeps a stack of directories before changing directories
# and the list that is returned is the stack of directories. You might see that
# the stack contains the notebooks directory, because that was our last working
# directory.
######################################################
#                RUN THIS CODE BLOCK                 #
#     THIS CODE BLOCK DOES NOT NEED TO BE EDITED     #
#  DO NOT CHANGE UNLESS YOU KNOW WHAT YOU'RE DOING!! #
######################################################

/home/staylor/lineage-growth


['~/lineage-growth/notebooks']

In [6]:
""" This file is completely user defined. We have provided a general starting point for the user to use as an example. """
import numpy as np
import scipy.stats as sp



class StateDistribution:
    def __init__(self, norm_loc, norm_scale):
        """ Initialization function should take in just in the parameters for the observations that comprise the multivariate random variable emission they expect their data to have. """
        self.norm_loc = norm_loc
        self.norm_scale = norm_scale
        self.params = [self.norm_loc, self.norm_scale]

    def rvs(self, size):  # user has to identify what the multivariate (or univariate if he or she so chooses) random variable looks like
        """ User-defined way of calculating a random variable given the parameters of the state stored in that observation's object. """
        # {
        norm_obs = sp.norm.rvs(loc=self.norm_loc, scale=self.norm_scale, size=size)  # normal observations
        #time_censor = [1] * len(gamma_obs)  # 1 if observed
        # } is user-defined in that they have to define and maintain the order of the multivariate random variables.
        # These tuples of observations will go into the cells in the lineage tree.
        list_of_tuple_of_obs = list(map(list, zip(norm_obs)))
        return list_of_tuple_of_obs

    def pdf(self, tuple_of_obs):  # user has to define how to calculate the likelihood
        """ User-defined way of calculating the likelihood of the observation stored in a cell. """
        # In the case of a univariate observation, the user still has to define how the likelihood is calculated,
        # but has the ability to just return the output of a known scipy.stats.<distribution>.<{pdf,pmf}> function.
        # In the case of a multivariate observation, the user has to decide how the likelihood is calculated.
        # In our example, we assume the observation's are uncorrelated across the dimensions (across the different
        # distribution observations), so the likelihood of observing the multivariate observation is just the product of
        # the individual observation likelihoods.

        norm_ll = sp.norm.pdf(tuple_of_obs[0], self.norm_loc, self.norm_scale)

        return norm_ll

    def estimator(self, list_of_tuples_of_obs, gammas):
        """ User-defined way of estimating the parameters given a list of the tuples of observations from a group of cells. """
        # unzipping the list of tuples
        unzipped_list_of_tuples_of_obs = list(zip(*list_of_tuples_of_obs))

        # getting the observations as individual lists
        # {
        try:
            norm_obs = list(unzipped_list_of_tuples_of_obs[0])
        except BaseException:
            norm_obs = []

        norm_loc_estimate, norm_scale_estimate = norm_estimator(norm_obs, gammas)

        state_estimate_obj = StateDistribution(norm_loc=norm_loc_estimate, norm_scale=norm_scale_estimate)
        # } requires the user's attention.
        # Note that we return an instance of the state distribution class, but now instantiated with the parameters
        # from estimation. This is then stored in the original state distribution object which then gets updated
        # if this function runs again.
        return state_estimate_obj

    def tHMM_E_init(self):
        """
        Initialize a default state distribution.
        """
        return StateDistribution(0, 1 * (np.random.uniform()))

    def __repr__(self):
        """
        Method to print out a state distribution object.
        """
        return "State object w/ parameters: {}, {}, {}.".format(self.norm_loc, self.norm_scale)


# Because parameter estimation requires that estimators be written or imported,
# the user should be able to provide
# estimators that can solve for the parameters that describe the distributions.
# We provide some estimators below as an example.
# Their use in the StateDistribution class is shown in the estimator class method.
# User must take care to define estimators that
# can handle the case where the list of observations is empty.


def norm_estimator(norm_obs, gammas):
    '''This function is an estimator for the mean and standard deviation of a normal distribution, including weighting for each state'''
    mu = (sum(gammas*norm_obs)/sum(gammas))
    std = sum(gammas*(norm_obs-mu)**2)/sum(gammas)
    
    return mu, std




In [7]:
from lineage.LineageTree import LineageTree
pi = np.array([0.6, 0.4], dtype="float")

T = np.array([[0.75, 0.25],
              [0.25, 0.75]], dtype="float")

In [8]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Small"
norm_loc0 = 13
norm_scale0 = 2.5

# State 1 parameters "Large"
norm_loc1 = 18
norm_scale1 = 4

state_obj0 = StateDistribution(norm_loc0, norm_scale0)
state_obj1 = StateDistribution(norm_loc1, norm_scale1)

E = [state_obj0, state_obj1]

In [9]:
lineage1 = LineageTree(pi, T, E, desired_num_cells=2**5 - 1)
# These are the minimal arguments required to instantiate lineages
print(lineage1)
print("\n")

IndexError: list index out of range